<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/utils-tests/10_root_finding/45_newton_raphson_complex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Add graph and math features
# 그래프, 수학 기능 추가
import pylab as py

# scipy.optimize.newton()
import scipy.optimize as so



In [ ]:
# symbolic processor
# 기호처리기
import sympy as sym
import sympy.utilities as su
sym.init_printing()



# 복소근과 뉴튼 랩슨법<br>Newton Rapson Method and Complex Roots



## A polynomial with complex roots<br>복소근을 갖는 다항식 예



The following video is about applying Newton Raphson method to find complex roots. (26m05s)<br>
아래 비디오는 뉴튼 랩슨법으로 복소근을 찾는 경우에 관한 것이다. (26m05s)



[![Newton's Fractal(which Newton know nothing about)](https://i.ytimg.com/vi/-RdOwhmqP5s/hqdefault.jpg)](https://youtu.be/-RdOwhmqP5s)



In [ ]:
z = sym.symbols('z', complex=True)



From the video, let's think about the following polynomial.<br>
영상의 여러 다항식 가운데 다음을 생각해 보자.



In [ ]:
P_z = z ** 5 + z ** 2 - z + 1
P_z



Its derivative would be as follows.<br>
그 미분은 다음과 같을 것이다.



In [ ]:
dP_dz = P_z.diff(z)
dP_dz



Let's make python functions.<br>파이썬 함수를 생성해 보자.



In [ ]:
p = su.lambdify(z, P_z)



In [ ]:
assert p(-1) == P_z.subs({z:-1})



In [ ]:
dp_dz = su.lambdify(z, dP_dz)



In [ ]:
assert dp_dz(-1) == dP_dz.subs({z:-1})



Let's visualize the complex function `p(z)`.<br>
해당 복소 함수 `p(z)` 를 시각화 해 보자.



In [ ]:
x = py.linspace(-3, 3, 150+1)
y = py.linspace(-2, 2, 100+1)
X, Y = py.meshgrid(x, y)
Z = X + Y * 1.0j
P = p(Z)



In [ ]:
cmap = "viridis"
levels = 32



Let's take a look at the real and imaginary parts of $P(z)$.<br>$P(z)$ 의 실수부와 허수부를 살펴보자.



In [ ]:
fig, ax = py.subplots(1, 2, figsize=(20, 5))
c0 = ax[0].pcolor(X, Y, P.real, cmap=cmap)
py.colorbar(c0, ax=ax[0])
ax[0].contour(X, Y, P.real, cmap="jet", levels=levels)
ax[0].axis("equal")
py.xlabel("$real(z)$")
py.ylabel("$imag(z)$")
py.title(r"$ real\left( P(z) \right) $");


c1 = ax[1].pcolor(X, Y, P.imag, cmap=cmap)
py.colorbar(c1, ax=ax[1])
ax[1].contour(X, Y, P.imag, cmap="jet", levels=levels)
ax[1].axis("equal");
py.xlabel("$real(z)$")
py.ylabel("$imag(z)$")
py.title(r"$ imag\left( P(z) \right) $");



What about the absolute values of $P(z)$?<br>$P(z)$ 의 절대값은 어떤가?



In [ ]:
log_abs_P = py.log(abs(P))



In [ ]:
fig, ax = py.subplots(figsize=(18, 10))

c_abs = ax.pcolor(X, Y, log_abs_P, cmap=cmap)
py.colorbar(c_abs, ax=ax)
ax.contour(X, Y, log_abs_P, cmap="jet", levels=levels)
ax.axis("equal");
py.xlabel("$real(z)$")
py.ylabel("$imag(z)$")
py.title(r"$ \left| P(z) \right| $");



## Finding complex roots using Newton-Raphson method<br>복소근을 뉴튼랩슨법으로 찾기



In [ ]:
class LogAttempts():
    def __init__(self):
        self.z_dict = {}
        self.z_list = []
    def init(self, z0:complex):
        self.z_dict[z0] = []
        self.z_list = self.z_dict[z0] = []
    def f(self, z:complex) -> complex:
        self.z_list.append(z)
        return p(z)



In [ ]:
logger = LogAttempts()



### Various initial conditions<br>다양한 초기 조건



In [ ]:
for z_initial in (-2.0 + 1.0j, -1.0 + 1.0j, 2.0j, 2.0 + 1.0j):

    logger.init(z_initial)
    f_z = logger.f

    root = so.newton(f_z, z_initial, fprime=dp_dz)



In [ ]:
fig, ax = py.subplots(figsize=(18, 10))

c_abs = ax.pcolor(X, Y, log_abs_P, cmap=cmap, alpha=0.75)
py.colorbar(c_abs, ax=ax)
ax.contour(X, Y, log_abs_P, cmap="jet", levels=levels)

for z_initial in logger.z_dict:
    z_array = py.array(logger.z_dict[z_initial])
    z_real = z_array.real
    z_imag = z_array.imag

    ax.plot(z_real, z_imag, '.-', label=f"{z_initial}")

ax.axis("equal");
ax.legend(loc=0)
py.xlabel("$real(z)$")
py.ylabel("$imag(z)$")
py.title(r"$ \left| P(z) \right| $");



## Another Example<br>다른 예



In [ ]:
r = 10.0



In [ ]:
Q_z = - (r * r - z * z) ** 0.5 + r * 0.5
Q_z



Its derivative would be as follows.<br>
그 미분은 다음과 같을 것이다.



In [ ]:
dQ_dz = Q_z.diff(z)
dQ_dz



In [ ]:
q = su.lambdify(z, Q_z)



In [ ]:
dq_dz = su.lambdify(z, dQ_dz)



In [ ]:
x = py.linspace(-r*2, r*2, 100+1)
y = py.linspace(-r, r, 100+1)
X, Y = py.meshgrid(x, y)
Z = X + Y * 1.0j
Q = q(Z)



In [ ]:
log_abs_Q = py.log(abs(Q))



In [ ]:
fig, ax = py.subplots(figsize=(18, 8))

c_abs = ax.pcolor(X, Y, log_abs_Q, cmap=cmap)
py.colorbar(c_abs, ax=ax)
ax.contour(X, Y, log_abs_Q, cmap="jet", levels=levels)
ax.axis("equal");
py.xlabel("$real(z)$")
py.ylabel("$imag(z)$")
py.title(r"$ \left| Q(z) \right| $");



In [ ]:
class LogAttemptsQ(LogAttempts):
    def __init__(self):
        # Understanding Python super() with __init__() methods, https://stackoverflow.com/questions/576169
        super(LogAttemptsQ, self).__init__()

    def f(self, z:complex) -> complex:
        self.z_list.append(z)
        result = q(z)
        assert 100 > abs(result), f"abs(result) = {abs(result)}"
        return result



In [ ]:
logger_q = LogAttemptsQ()



In [ ]:
for z_initial in (-1.5 * r + 0.01j, -0.5 * r + 1.0j, 0.5 * r + 1.0j):

    logger_q.init(z_initial)
    f_z = logger_q.f

    try:
        root = so.newton(f_z, z_initial, fprime=dq_dz)
    except RuntimeError as e:
        print(e)



In [ ]:
fig, ax = py.subplots(figsize=(18, 8))

c_abs = ax.pcolor(X, Y, log_abs_Q, cmap=cmap, alpha=0.75)
py.colorbar(c_abs, ax=ax)
ax.contour(X, Y, log_abs_Q, cmap="jet", levels=levels)

for z_initial in logger_q.z_dict:
    z_array = py.array(logger_q.z_dict[z_initial])
    z_real = z_array.real
    z_imag = z_array.imag

    ax.plot(z_real, z_imag, '.-', label=f"{z_initial}")

ax.axis("equal");
ax.legend(loc=0)
py.xlabel("$real(z)$")
py.ylabel("$imag(z)$")
py.title(r"$ \left| Q(z) \right| $");



## Final Bell<br>마지막 종



In [ ]:
# stackoverfow.com/a/24634221
import os
os.system("printf '\a'");

